In [1]:
%load_ext autoreload
%autoreload 2

# Key and Ciphertext saving

Load packages 

In [2]:
from time import time
import numpy as np
import fase
from fase import HEAAN as he
from fase.core.common import HEAANContext

Using CPU version HEAAN


Set up FHE context.  
Secret key is automatically stored.

In [47]:
logq = 120
logp = 30
logn = 4 # small logn for quick test
n = 1*2**logn
slots = n

t0 = time()
hec = HEAANContext(logn, logp, logq, boot=False, 
                   key_path="./", rot_l=[1,2])
print(f"Generating HEAAN context took {time() - t0} seonds")
print(f"logn  {logn},  logp {logp},  logq{logq}")

saving secret key done.
HEAAN CKKS setup is ready 
Generating HEAAN context took 9.487056732177734 seonds
logn  4,  logp 30,  logq120


Generate a ctxt and save it to a file

In [55]:
def compare(expected, calculated, eps=1e-4):
    print("Two arrays are close enough", np.all(np.isclose(expected, calculated[:len(expected)], atol=eps)))

In [50]:
val = np.random.rand(n)
ctxt = hec.encrypt(val)

he.SerializationUtils.writeCiphertext(ctxt, "ctxt.dat")

decrypted = hec.decrypt(ctxt)

In [56]:
# encrypted and decrypted value
compare(val, decrypted, eps=1e-4) # But, isn't it too large an error??

Two arrays are close enough True


==> First of all, encryption and decryption works.

In [57]:
# Load the secret key and try decrypting
hec2 = HEAANContext(logn, logp, logq, boot=False, is_owner=True, load_sk=True)
compare(val, hec2.decrypt(ctxt))

HEAAN CKKS setup is ready loading secret key done.

Two arrays are close enough True


Loaded secret key works as expected.

## Rotation

In [64]:
hec.lrot(ctxt, 1, inplace=True)

compare(hec.decrypt(ctxt), np.roll(val, -1))

Two arrays are close enough False


### Multilply two ctxts

In [67]:
val = np.random.rand(n)
val2 = np.arange(n)

# FHE
ctxt = hec.encrypt(val)
ctxt2 = hec.encrypt(val2)
ctxt3 = hec.mult(ctxt, ctxt2, inplace=False)

# ordinary
val *= val2

compare(hec.decrypt(ctxt3), val)

Two arrays are close enough True


### Multiply with plain text

In [71]:
ctxt4 = hec.multByVec(ctxt, val2, inplace=False)

compare(hec.decrypt(ctxt4), val)

Two arrays are close enough True
